# Chapter 6

In [1]:
import pandas as pd

# 6  Data Loading, Storage, and File Formats

#### Reading data and making it accessible (often called data loading)
#### The term parsing is also sometimes used to describe loading text data and interpreting it as tables and different data types. 

# 6.1 Reading and Writing Data in Text Format

In [2]:
# Table 6.1: Text and binary data loading functions in pandas
Function	Description
read_csv:	Load delimited data from a file, URL, or file-like object; use comma as default delimiter
read_fwf:	Read data in fixed-width column format (i.e., no delimiters)
read_clipboard:	Variation of read_csv that reads data from the clipboard; useful for converting tables from web pages
read_excel:	Read tabular data from an Excel XLS or XLSX file
read_hdf:	Read HDF5 files written by pandas
read_html:	Read all tables found in the given HTML document
read_json:	Read data from a JSON (JavaScript Object Notation) string representation, file, URL, or file-like object
read_feather:	Read the Feather binary file format
read_orc:	Read the Apache ORC binary file format
read_parquet:	Read the Apache Parquet binary file format
read_pickle:	Read an object stored by pandas using the Python pickle format
read_sas:	Read a SAS dataset stored in one of the SAS system's custom storage formats
read_spss:	Read a data file created by SPSS
read_sql:	Read the results of a SQL query (using SQLAlchemy)
read_sql_table:	Read a whole SQL table (using SQLAlchemy); equivalent to using a query that selects everything in that table using read_sql
read_stata:	Read a dataset from Stata file format
read_xml:	Read a table of data from an XML file

SyntaxError: invalid syntax (4272952187.py, line 2)

## Indexing

#### Can treat one or more columns as the returned DataFrame, and whether to get column names from the file, arguments you provide, or not at all. 

## Type inference and data conversion

#### Includes the user-defined value conversions and custom list of missing value markers.

## Date and time parsing

#### Includes a combining capability, including combining date and time information spread over multiple columns into a single column in the result.

## Iterating

#### Support for iterating over chunks of very large files.

## Unclean data issues

#### Includes skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

#### Some of these functions perform type inference, because the column data types are not part of the data format. That means you don’t necessarily have to specify which columns are numeric, integer, Boolean, or string. Other data formats, like HDF5, ORC, and Parquet, have the data type information embedded in the format.

In [3]:
# CSV text file
!cat examples/ex1.csv
a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

# Since this is comma-delimited, we can then use pandas.read_csv to read it into a DataFrame:
df = pd.read_csv("examples/ex1.csv")
df

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

NameError: name 'a' is not defined

In [ ]:
# A file will not always have a header row. Consider this file:
!cat examples/ex2.csv
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

# To read this file, you have a couple of options. You can allow pandas to assign default column names, or you can specify names yourself:
pd.read_csv("examples/ex2.csv", header=None)

In [4]:
pd.read_csv("examples/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
# Suppose you wanted the message column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named "message" using the index_col argument:
names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [6]:
# If you want to form a hierarchical index from multiple columns, pass a list of column numbers or names:
!cat examples/csv_mindex.csv

parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [7]:
# A table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:
!cat examples/ex3.txt


# You can pass a regular expression as a delimiter for pandas.read_csv. This can be expressed by the regular expression \s+:
result = pd.read_csv("examples/ex3.txt", sep="\s+")
result

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [8]:
# The file parsing functions have many additional arguments to help you handle the wide variety of exception file formats that occur
!cat examples/ex4.csv
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
# Handling missing values is an important and frequently nuanced part of the file reading process.
# Missing data is usually either not present (empty string) or marked by some sentinel (placeholder) value.
!cat examples/ex5.csv
result = pd.read_csv("examples/ex5.csv")
result

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [10]:
# Recall that pandas outputs missing values as NaN, so we have two null or missing values in result:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [11]:
# The na_values option accepts a sequence of strings to add to the default list of strings recognized as missing:
result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [12]:
# pandas.read_csv has a list of many default NA value representations, but these defaults can be disabled with the keep_default_na option:
result2 = pd.read_csv("examples/ex5.csv", keep_default_na=False)
result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [13]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [14]:
result3 = pd.read_csv("examples/ex5.csv", keep_default_na=False,
                      na_values=["NA"])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [15]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [16]:
# Different NA sentinels can be specified for each column in a dictionary:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("examples/ex5.csv", na_values=sentinels,
            keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


In [17]:
# Table 6.2: Some pandas.read_csv function arguments
Argument	Description
path:	String indicating filesystem location, URL, or file-like object.
sep or delimiter:	Character sequence or regular expression to use to split fields in each row.
header:	Row number to use as column names; defaults to 0 (first row), but should be None if there is no header row.
index_col:	Column numbers or names to use as the row index in the result; can be a single name/number or a list of them for a hierarchical index.
names:	List of column names for result.
skiprows:	Number of rows at beginning of file to ignore or list of row numbers (starting from 0) to skip.
na_values:	Sequence of values to replace with NA. They are added to the default list unless keep_default_na=False is passed.
keep_default_na:	Whether to use the default NA value list or not (True by default).
comment	Character(s): to split comments off the end of lines.
parse_dates:	Attempt to parse data to datetime; False by default. If True, will attempt to parse all columns. Otherwise, can specify a list of column numbers or names to parse. If element of list is tuple or list, will combine multiple columns together and parse to date (e.g., if date/time split across two columns).
keep_date_col:	If joining columns to parse date, keep the joined columns; False by default.
converters:	Dictionary containing column number or name mapping to functions (e.g., {"foo": f} would apply the function f to all values in the "foo" column).
dayfirst:	When parsing potentially ambiguous dates, treat as international format (e.g., 7/6/2012 -> June 7, 2012); False by default.
date_parser:	Function to use to parse dates.
nrows:	Number of rows to read from beginning of file (not counting the header).
iterator:	Return a TextFileReader object for reading the file piecemeal. This object can also be used with the with statement.
chunksize:	For iteration, size of file chunks.
skip_footer:	Number of lines to ignore at end of file.
verbose:	Print various parsing information, like the time spent in each stage of the file conversion and memory use information.
encoding:	Text encoding (e.g., "utf-8 for UTF-8 encoded text). Defaults to "utf-8" if None.
squeeze:	If the parsed data contains only one column, return a Series.
thousands:	Separator for thousands (e.g., "," or "."); default is None.
decimal:	Decimal separator in numbers (e.g., "." or ","); default is ".".
engine:	CSV parsing and conversion engine to use; can be one of "c", "python", or "pyarrow". The default is "c", though the newer "pyarrow" engine can parse some files much faster. The "python" engine is slower but supports some features that the other engines do not.

SyntaxError: invalid syntax (2096339495.py, line 2)

## Reading Text Files in Pieces

In [18]:
pd.options.display.max_rows = 10
result = pd.read_csv("examples/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [19]:
# To read a file in pieces, specify a chunksize as a number of rows:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
type(chunker)

pandas.io.parsers.readers.TextFileReader

In [20]:
# The TextFileReader object returned by pandas.read_csv allows you to iterate over the parts of the file according to the chunksize
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

#### TextFileReader is also equipped with a get_chunk method that enables you to read pieces of an arbitrary size.

## Writing Data to Text Format

In [21]:
data = pd.read_csv("examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [22]:
# Using DataFrame’s to_csv method, we can write the data out to a comma-separated file:
data.to_csv("examples/out.csv")
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [23]:
# Other delimiters can be used, of course (writing to sys.stdout so it prints the text result to the console rather than a file):

import sys

data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [24]:
# Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value:
data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [25]:
# With no other options specified, both the row and column labels are written. Both of these can be disabled:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [26]:
# You can also write only a subset of the columns, and in an order of your choosing:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


## Working with Other Delimited Formats

In [27]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [28]:
# For any file with a single-character delimiter, you can use Python’s built-in csv module. # To use it, pass any open file or file-like object to csv.reader:
import csv

f = open("examples/ex7.csv")
reader = csv.reader(f)

# Iterating through the reader like a file yields lists of values with any quote characters removed:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


In [29]:
f.close()

# From there, it’s up to you to do the wrangling necessary to put the data in the form that you need. Let's take this step by step. First, we read the file into a list of lines:
with open("examples/ex7.csv") as f:
    lines = list(csv.reader(f))

# Then we split the lines into the header line and the data lines:
header, values = lines[0], lines[1:]

# Then we can create a dictionary of data columns using a dictionary comprehension and the expression zip(*values) (beware that this will use a lot of memory on large files), which transposes rows to columns:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [30]:
# To define a new format with a different delimiter, string quoting convention, or line terminator, we could define a simple subclass of csv.Dialect:
class my_dialect(csv.Dialect):
    lineterminator = "\n"
    delimiter = ";"
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL
reader = csv.reader(f, dialect=my_dialect)

ValueError: I/O operation on closed file.

In [31]:
reader = csv.reader(f, delimiter="|")

ValueError: I/O operation on closed file.

In [33]:
# Table 6.3: CSV dialect options
Argument	Description
delimiter:	One-character string to separate fields; defaults to ",".
lineterminator:	Line terminator for writing; defaults to "\r\n". Reader ignores this and recognizes cross-platform line terminators.
quotechar:	Quote character for fields with special characters (like a delimiter); default is '"'.
quoting:	Quoting convention. Options include csv.QUOTE_ALL (quote all fields), csv.QUOTE_MINIMAL (only fields with special characters like the delimiter), csv.QUOTE_NONNUMERIC, and csv.QUOTE_NONE (no quoting). See Python’s documentation for full details. Defaults to QUOTE_MINIMAL.
skipinitialspace:	Ignore whitespace after each delimiter; default is False.
doublequote:	How to handle quoting character inside a field; if True, it is doubled (see online documentation for full detail and behavior).
escapechar:	String to escape the delimiter if quoting is set to csv.QUOTE_NONE; disabled by default.

SyntaxError: invalid syntax (3455713724.py, line 2)

#### For files with more complicated or fixed multicharacter delimiters, you will not be able to use the csv module. In those cases, you’ll have to do the line splitting and other cleanup using the string’s split method or the regular expression method re.split.

In [34]:
# To write delimited files manually, you can use csv.writer. It accepts an open, writable file object and the same dialect and format options as csv.reader:
with open("mydata.csv", "w") as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(("one", "two", "three"))
    writer.writerow(("1", "2", "3"))
    writer.writerow(("4", "5", "6"))
    writer.writerow(("7", "8", "9"))

## JSON Data

#### JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. 

In [35]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

#### JSON is very nearly valid Python code with the exception of its null value null and some other nuances (such as disallowing trailing commas at the end of lists). 

#### The basic types are objects (dictionaries), arrays (lists), strings, numbers, Booleans, and nulls. All of the keys in an object must be strings. 

#### There are several Python libraries for reading and writing JSON data. 

#### To convert a JSON string to Python form, use json.loads:

In [36]:
import json

result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

In [37]:
# json.dumps, on the other hand, converts a Python object back to JSON:

asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

In [38]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

,name,age
0,Scott,34
1,Katie,42


In [39]:
# The pandas.read_json can automatically convert JSON datasets in specific arrangements into a Series or DataFrame.
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [40]:
data = pd.read_json("examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [41]:
# If you need to export data from pandas to JSON, one way is to use the to_json methods on Series and DataFrame:
data.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [42]:
data.to_json(sys.stdout, orient="records")

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

## XML and HTML: Web Scraping

#### pandas has a built-in function, pandas.read_html, which uses all of these libraries to automatically parse tables out of HTML files as DataFrame objects. 

In [34]:
sudo pip3 install lxml

SyntaxError: invalid syntax (2149339370.py, line 1)

In [44]:
tables = pd.read_html("examples/fdic_failed_bank_list.html")
len(tables)

1

In [45]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


In [46]:
# Because failures has many columns, pandas inserts a line break character \.

close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

Closing Date
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 15, dtype: int64

## Parsing XML with lxml.objectify


In [47]:
<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>

SyntaxError: invalid syntax (2722320822.py, line 1)

In [48]:
# Using lxml.objectify, we parse the file and get a reference to the root node of the XML file with getroot:
from lxml import objectify

path = "datasets/mta_perf/Performance_MNR.xml"

with open(path) as f:
    parsed = objectify.parse(f)

root = parsed.getroot()

data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/mta_perf/Performance_MNR.xml'

In [49]:
# Convert this list of dictionaries into a DataFrame:
perf = pd.DataFrame(data)
perf.head()

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [50]:
# pandas's pandas.read_xml function turns this process into a one-line expression:
perf2 = pd.read_xml(path)
perf2.head()

XMLSyntaxError: Start tag expected, '<' not found, line 1, column 1 (<string>, line 1)

# 6.2 Binary Data Formats

#### One simple way to store (or serialize) data in binary format is using Python’s built-in pickle module. 

#### pandas objects all have a to_pickle method that writes the data to disk in pickle format:

In [51]:
frame = pd.read_csv("examples/ex1.csv")
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [52]:
frame.to_pickle("examples/frame_pickle")

#### Pickle files are in general readable only in Python. You can read any "pickled" object stored in a file by using the built-in pickle directly, or even more conveniently using pandas.read_pickle:


In [53]:
pd.read_pickle("examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [55]:
fec = pd.read_parquet('datasets/fec/fec.parquet')

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/fec/fec.parquet'

## Reading Microsoft Excel Files

In [37]:
pip install openpyxl xlrd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 4.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [38]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

# This object can show you the list of available sheet names in the file:
xlsx.sheet_names

['Sheet1']

In [39]:
# Data stored in a sheet can then be read into DataFrame with parse:
xlsx.parse(sheet_name="Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [40]:
# This Excel table has an index column, so we can indicate that with the index_col argument:
xlsx.parse(sheet_name="Sheet1", index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [41]:
frame = pd.read_excel("examples/ex1.xlsx", sheet_name="Sheet1")
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [43]:
# To write pandas data to Excel format, you must first create an ExcelWriter, then write data to it using the pandas object's to_excel method:

writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.close()

# You can also pass a file path to to_excel and avoid the ExcelWriter:
frame.to_excel("examples/ex2.xlsx")

## Using HDF5 Format

In [45]:
# To get started with HDF5 and pandas, you must first install PyTables by installing the tables package with conda:

In [46]:
pip install tables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 13.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      ld: library not found for -lhdf5
      clang: error: linker command failed with exit code 1 (use -v to see invocation)
      <string>:19: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      cpuinfo failed, assuming no CPU features: 'flags'
      * Using Python 3.8.18 | packaged by conda-forge | (default, Oct 10 2023, 15:46:56)
      * Found cython 3.0.6
      * USE_PKGCONFIG: False
      * Found conda env: ``/Users/joey/Library/jupyterlab-desktop/jlab_server``
      .. ERROR:: Could not find a local HDF5 installation.
         You may need to explicitly state where your local HDF5 head

In [51]:
# The HDFStore class works like a dictionary and handles the low-level details:
frame = pd.DataFrame({"a": np.random.standard_normal(100)})
store = pd.HDFStore("examples/mydata.h5")

store["obj1"] = frame
store["obj1_col"] = frame["a"]
store

ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

In [52]:
store["obj1"]

UsageError: Unknown variable '[obj1]'


In [53]:
# HDFStore supports two storage schemas, "fixed" and "table" (the default is "fixed"). 
store.put("obj2", frame, format="table")
store.select("obj2", where=["index >= 10 and index <= 15"])

NameError: name 'store' is not defined

In [54]:
frame.to_hdf("examples/mydata.h5", "obj3", format="table")
pd.read_hdf("examples/mydata.h5", "obj3", where=["index < 5"])

ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

In [55]:
import os
os.remove("examples/mydata.h5")

FileNotFoundError: [Errno 2] No such file or directory: 'examples/mydata.h5'

# 6.3 Interacting with Web APIs


In [56]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [58]:
import requests

url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.raise_for_status()
resp

<Response [200]>

In [59]:
# Practice to always call raise_for_status after using requests.get to check for HTTP errors.

# The response object’s json method will return a Python object containing the parsed JSON data as a dictionary or list (depending on what JSON is returned):


In [60]:
data = resp.json()
data[0]["title"]

'BUG: calendar day date_range AmbiguousTimeError with unambiguous DST boundaries'

#### Each element in data is a dictionary containing all of the data found on a GitHub issue page (except for the comments). We can pass data directly to pandas.DataFrame and extract fields of interest:

In [61]:
issues = pd.DataFrame(data, columns=["number", "title", "labels", "state"])
issues

,number,title,labels,state
0,56240,BUG: calendar day date_range AmbiguousTimeErro...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
1,56238,BUG: Read CSV on python engine fails with call...,[],open
2,56237,DEPR: Default of observed=False in DataFrame.p...,"[{'id': 13098779, 'node_id': 'MDU6TGFiZWwxMzA5...",open
3,56236,DEPR: observed=False default for DataFrame.piv...,"[{'id': 13098779, 'node_id': 'MDU6TGFiZWwxMzA5...",open
4,56235,Table cell styling,[],open
...,...,...,...,...
25,56188,Adjust tests in array folder for new string op...,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
26,56187,BUG: value_counts not preserving object dtype,"[{'id': 31404521, 'node_id': 'MDU6TGFiZWwzMTQw...",open
27,56184,Adjust tests in root directory for new string ...,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open
28,56183,BUG: mode not preserving object dtype for stri...,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2OD...",open


# 6.4 Interacting with Databases

In [62]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 REAL,        d INTEGER
 );"""

con = sqlite3.connect("mydata.sqlite")
con.execute(query)

In [64]:
con.commit()

data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)

In [65]:
con.commit()
cursor = con.execute("SELECT * FROM test")
rows = cursor.fetchall()

rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [66]:
# You can pass the list of tuples to the DataFrame constructor, but you also need the column names, contained in the cursor’s description attribute.
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,REAL,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [67]:
pip install sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [68]:
# pandas has a read_sql function that enables you to read data easily from a general SQLAlchemy connection.
import sqlalchemy as sqla

db = sqla.create_engine("sqlite:///mydata.sqlite")
pd.read_sql("SELECT * FROM test", db)

,a,b,REAL,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
